In [1]:
from wq_api import *

session = sign_in(username,password)

<Response [201]>


In [2]:
# data is restored in {index}_refine_1.csv, where index is generated by order
import os

dir_path = './refine_data'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

index = 0
while True:
    to_path = f'{dir_path}/{index}_refine_1.csv'
    if not os.path.exists(to_path):
        break
    index += 1

print(f'Next available file path: {to_path}')

Next available file path: ./refine_data/0_refine_1.csv


In [3]:
setting = {
    'universe':['TOP3000', 'MINVOL1M'],
    'decay':[1,4,6,10,20,25,30,60],
    'neutralization':['MARKET','INDUSTRY','SUBINDUSTRY','SECTOR','COUNTRY']
}

In [5]:
# make alpha template
alphas = []

alpha = "regression_neut(vector_neut(ts_rank(vec_max({})/close,120),ts_median(cap, 120) ),abs(ts_mean({},252)/ts_std_dev({},252)))" \
        .format('anl9_s_numupunfiltered','returns','returns')
alphas.append(alpha)

In [6]:
# make test alpha list
priority_alpha_list=[]
priority_alpha_list = [
    {
        'type': 'REGULAR',
        'settings': {
            'instrumentType': 'EQUITY',
            'region': 'GLB',
            'universe': universe,
            'delay': 1,
            'decay': decay,
            'neutralization': neutralization,
            'truncation': 0.08,
            'pasteurization': 'ON',
            'unitHandling': 'VERIFY',
            'nanHandling': 'ON',
            'language': 'FASTEXPR',
            'visualization': False,
        },
        'regular': alpha
    }
    for alpha in alphas
    for universe in setting['universe']
    for decay in setting['decay']
    for neutralization in setting['neutralization']
]
    # simulation_response = session.post('https://api.worldquantbrain.com/simulations', json=simulation_data)

In [7]:
import pandas as pd
# 将 priority_alpha_list 转换为 DataFrame
priority_alpha_df = pd.DataFrame(priority_alpha_list)

# 将 settings 列中的字典展开为单独的列
settings_df = priority_alpha_df['settings'].apply(pd.Series)

# 将展开的 settings 列与其他列合并
priority_alpha_df = pd.concat([priority_alpha_df.drop(columns=['settings']), settings_df], axis=1)

print(priority_alpha_df)

priority_alpha_df.to_csv(to_path)

        type                                            regular  \
0    REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
1    REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
2    REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
3    REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
4    REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
..       ...                                                ...   
315  REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
316  REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
317  REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
318  REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   
319  REGULAR  regression_neut(vector_neut(ts_rank(vec_max(an...   

    instrumentType region  universe  delay  decay neutralization  truncation  \
0           EQUITY    GLB   TOP3000      1      1         MARKET        0.08   
1           EQUITY    GLB   TOP3000